In [ ]:
using Random
using DelimitedFiles: writedlm, readdlm
using Statistics

using ProgressBars
using ProgressMeter

In [ ]:
function erdos_renyi(N::Int,p::Float64,seed::Int)
    rng = MersenneTwister(seed)
    nl = Dict{Int,Vector{Int32}}()
    for i in 1:N 
        nl[i] = []
    end
    nodes = Int32.(collect(1:N))
    @showprogress for i in 1:N
        for j in i:N
            if rand(rng) < p
                push!(nl[i],j)
                push!(nl[j],i)
            end
        end
    end
    nl
end

In [ ]:
function oscillators_a(seed1::Int,seed2::Int)
    rng = MersenneTwister(seed1)

    p = 0.02

    N = 300
    ϵ = 1e-2
    δt = 1e-4

    nl = erdos_renyi(N,p,seed2)

    θ = zeros(N)
    θ_0 = rand(rng,N)

    θ .+= θ_0

    T = 10^7

    t1 = []
    M = []

    @showprogress for t in 1:T
        θ .+= δt
        idx_threshold = findall(θ .>= 1) # Índices i de los elementos de θ en los que que θ_i >= θ_thr
        for i in idx_threshold
            θ[nl[i]] += ϵ*θ[nl[i]]
        end
        θ[idx_threshold] .= 0
        if 1 in idx_threshold
            idx_upper = findall(θ .>= 0.5)
            m_upper = mean(1 .- θ[idx_upper])
            idx_lower = findall(θ .< 0.5)
            m_lower = mean(θ[idx_lower])
            push!(M,1 - (m_upper+m_lower))
            push!(t1,δt*t)
        end
    end
    data = hcat(t1,M)
    fname = "1a/osci_$(seed1)_$(seed2).txt"
    open(fname,"w") do io
        writedlm(io,data)
    end
end